In this section, I will progress from the exploratory data analysis to creating a predictive model that forecasts monthly total sales per region.

Modeling Steps taken:
1. Aggregate Data: Convert daily sales into monthly totals to reduce noise and capture seasonal patterns.
2. Feature Engineering: Create time-based features such as lag values (sales from previous months), rolling averages, and calendar features (month, year).
3. Train & Test Split: – Use a time-based split so the model is trained on past data and tested on unseen future months.
4. Model Training: Fit a Random Forest Regressor to learn patterns and relationships from historical sales trends.
5. Evaluation: Compare the model’s performance to a naive baseline (last month’s sales) using metrics like MAE, RMSE, and MAPE.
6. Forecasting: Generate predictions for future months using the trained model.

This approach allows us to leverage historical patterns to make accurate, actionable forecasts for sales planning and decision-making.

In [1]:
#Import Libraries

from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

In [2]:
#Load the raw data
data_path = Path("..") / "data" / "data_raw" / "train.csv"
df = pd.read_csv(data_path, parse_dates=['Order Date'])  #Ensure 'Order Date' becomes datetime

C:\Users\Jaime\anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1063: UserWarning: Parsing '15/04/2018' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\Users\Jaime\anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1063: UserWarning: Parsing '22/11/2016' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\Users\Jaime\anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1063: UserWarning: Parsing '13/05/2015' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\Users\Jaime\anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1063: UserWarning: Parsing '27/08/2015' in DD/MM/YYYY format. Provide format or specif

In [6]:
#Ensure columns needed are included, replace missing values
expected_cols = {'Order Date', 'Sales', 'Region', 'Postal Code'}
missing = expected_cols - set(df.columns)
if missing:
    raise ValueError(f"Missing required column(s): {missing}")

print(f"Raw rows: {len(df)}")
print("Date range:", df['Order Date'].min(), "→", df['Order Date'].max())

if missing:
    raise ValueError(f"Missing required column(s): {missing}")


Raw rows: 9800
Date range: 2015-01-02 00:00:00 → 2018-12-30 00:00:00


In [8]:
#Add a 'Month' column for grouping
df['Month'] = df['Order Date'].dt.to_period('M').dt.to_timestamp()

In [14]:
#Aggregate to monthly total sales per Region + postal code and sales

monthly = (
    df.groupby(['Month', 'Region', 'Postal Code'])['Sales']
      .sum()
      .reset_index()
      .sort_values(['Month', 'Region', 'Postal Code'])
)

# Format Postal Code as integer (no decimal)
monthly['Postal Code'] = monthly['Postal Code'].astype(int)

# Format Sales as currency without decimals
monthly['Sales'] = monthly['Sales'].map(lambda x: f"${x:,.0f}")

display(monthly.head())

,Month,Region,Postal Code,Sales
0,2015-01-01,Central,48185,$709
1,2015-01-01,Central,48234,$445
2,2015-01-01,Central,53711,"$2,854"
3,2015-01-01,Central,54302,$469
4,2015-01-01,Central,55044,"$3,746"


In [1]:
import sys
sys.executable

'C:\\Users\\Jaime\\DTSC 670\\.venv\\Scripts\\python.exe'